# Fidelity Check

We can use the theoretical results to validate calculations. The formulas are available at https://quantumcomputing.stackexchange.com/questions/16074/how-to-calculate-the-average-fidelity-of-an-amplitude-damping-channel 

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info.states.utils import _format_state, partial_trace
from qiskit.quantum_info.states.densitymatrix import Statevector
import numpy as np
import math
from numpy import pi

In [3]:
# How to check if the Fidelity is right?
# We'll compare against the theorical one.
# We have the theoretical values in the previous cell - see the picture or check the URL
# We provide as initial states the amplitudes corresponfing to |0>, |1>, |+> and |-> 
initial_state_0=[1,0,1/math.sqrt(2),1/math.sqrt(2)/1]
initial_state_1=[0,1,1/math.sqrt(2),-1/math.sqrt(2)/1]

In [12]:
# We simulate for a value of lambda (attenuation in the amplitude damping channel)
lambda_ = 0.1
eta = np.arcsin(np.sqrt(lambda_))

qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circ = QuantumCircuit(qreg_q, creg_c)
backend_sim = Aer.get_backend('statevector_simulator')
stavec = []
fidelity = []


print("lambda =" + str(format(lambda_)))
print("-----------------------------------")

for i in range(len(initial_state_0)):

    # Here we present the initial state and the theoretical fidelity
    print("Initial State " + str(format(initial_state_0[i])) + "x|0> + " + str(format(initial_state_1[i])) + "x|1>")
    if i== 0:
        print("Theoretical fidelity = " +'\033[1m' + "1")
        print ('\033[0m')
    elif i== 1:
        print("Theoretical fidelity = 1 - lambda = " + '\033[1m' + str(format(1-lambda_)))
        print ('\033[0m')
    else:
        print("Theoretical fidelity = 1/2 + sqrt(1-lambda)/2 = " + '\033[1m' + str(format(1/2 + np.sqrt(1-lambda_)/2)))
        print ('\033[0m')
    
    # Here we use the circuit to obtain the final state and then calculate the fidelity 
    circ.initialize([initial_state_0[i], initial_state_1[i]], qreg_q[0])
    circ.reset(qreg_q[1])
    circ.cry(2*eta, qreg_q[0], qreg_q[1])
    circ.cx(qreg_q[1], qreg_q[0])
    circ.rx(0, qreg_q[0])
    circ.ry(0, qreg_q[0])
    res = execute(circ, backend_sim).result()
    sv = res.get_statevector(circ)
    stavec.append([sv[0], sv[1]])
    
    # Here is the MISTAKE. The statevector for the 1st qubit can't be obtained directly from the 1st and 2nd component of the total statevector (sv)
    # This is also the reason why the error "quantum state not valid" appears sometimes
    print ("output state =" + str(format(sv[0])) + "x|0> + " + str(format(sv[1])) + "x|1>")
    print(state_fidelity([initial_state_0[i], initial_state_1[i]], [sv[0], sv[1]], validate=False))
    state1 = _format_state([initial_state_0[i], initial_state_1[i]], validate=False)
    state2 = _format_state([sv[0], sv[1]], validate=False)
    if isinstance(state2, list):
        state2 = np.array(state, dtype=complex)
    if isinstance(state2, np.ndarray):
        ndim = state.ndim
        print(ndim)
        if ndim == 1:
            state = Statevector(state2)
        elif ndim == 2:
            dim1, dim2 = state2.shape
            if dim2 == 1:
                state = Statevector(state2)
            elif dim1 == dim2:
                state = DensityMatrix(state2)
    if not isinstance(state2, Statevector):
        print("Input is not a quantum state")
    if True and not state2.is_valid():
        print("Input quantum state is not a valid")

    arr1 = state1.data
    arr2 = state2.data
#    if isinstance(state1, Statevector):
#        if isinstance(state2, Statevector):
            # Fidelity of two Statevectors
    fid = np.abs(arr2.conj().dot(arr1))**2
    print("our calculation =" + '\033[91m' + str(format(float(np.real(fid)))) + '\033[90m')
    print("-----------------------------------")
    
    # To get the 1st qubit final state (densitymatrix) we can take for the first qubit by taking the partial trace
    # We print it to see how this process work
    print(np.outer(sv,sv))
    partial_density_matrix = partial_trace(np.outer(sv,sv), [1])
    print(partial_density_matrix)
    print('\033[1m')
    print(state_fidelity([initial_state_0[i], initial_state_1[i]], partial_density_matrix, validate=True))
    print ('\033[0m')
    print("-----------------------------------")
    

lambda =0.1
-----------------------------------
Initial State 1x|0> + 0x|1>
Theoretical fidelity = 1

output state =(1+0j)x|0> + 0jx|1>
1.0
our calculation =1.0
-----------------------------------
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
DensityMatrix([[1.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j]],
              dims=(2,))

1.0

-----------------------------------
Initial State 0x|0> + 1x|1>
Theoretical fidelity = 1 - lambda = 0.9

output state =0jx|0> + (0.9486832980505138+0j)x|1>
0.8999999999999999
Input quantum state is not a valid
our calculation =0.8999999999999999
-----------------------------------
[[0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0.9+0.j 0.3+0.j 0. +0.j]
 [0. +0.j 0.3+0.j 0.1+0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]]
DensityMatrix([[0.1+0.j, 0. +0.j],
               [0. +0.j, 0.9+0.j]],
              dims=(2,))

0.8999999999999999

-----------------------------------
Init